# Artificial Neural Networks and Deep Learning

---

## Homework 2: Minimal Working Example

To make your first submission, follow these steps:
1. Create a folder named `[2024-2025] AN2DL/Homework 2` in your Google Drive.
2. Upload the `mars_for_students.npz` file to this folder.
3. Upload the Jupyter notebook `Homework 2 - Minimal Working Example.ipynb`.
4. Load and process the data.
5. Implement and train your model.
6. Submit the generated `.csv` file to Kaggle.


## 🌐 Connect Colab to Google Drive

In [1]:
from google.colab import drive

drive.mount("/gdrive")
%cd /gdrive/My Drive/[2024-2025] AN2DL/Homework 2

Mounted at /gdrive
/gdrive/My Drive/[2024-2025] AN2DL/Homework 2


## ⚙️ Import Libraries

In [2]:
import os
from datetime import datetime

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from tensorflow.keras import models
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {tfk.__version__}")
print(f"GPU devices: {len(tf.config.list_physical_devices('GPU'))}")

TensorFlow version: 2.17.1
Keras version: 3.5.0
GPU devices: 1


## ⏳ Load the Data

In [3]:
data = np.load("mars_for_students.npz")

training_set = data["training_set"]
X_train = training_set[:, 0]
y_train = training_set[:, 1]

X_test = data["test_set"]

print(f"Training X shape: {X_train.shape}")
print(f"Training y shape: {y_train.shape}")
print(f"Test X shape: {X_test.shape}")

# Separiamo il train set in training e validation set
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42  # 20% per il validation set
)

Training X shape: (2615, 64, 128)
Training y shape: (2615, 64, 128)
Test X shape: (10022, 64, 128)


## 🛠️ Train and Save the Model

In [4]:
# Add color channel and rescale pixels between 0 and 1
X_train = X_train[..., np.newaxis] / 255.0
###
X_val = X_val[..., np.newaxis] / 255.0
X_test = X_test[..., np.newaxis] / 255.0

input_shape = X_train.shape[1:]
num_classes = len(np.unique(y_train))

# Verifica della forma dei dati
print(f"Training X shape: {X_train.shape}")
print(f"Training y shape: {y_train.shape}")
print(f"Validation X shape: {X_val.shape}")
print(f"Validation y shape: {y_val.shape}")
print(f"Test X shape: {X_test.shape}")

print(f"Input shape: {input_shape}")
print(f"Number of classes: {num_classes}")

Training X shape: (2092, 64, 128, 1)
Training y shape: (2092, 64, 128)
Validation X shape: (523, 64, 128, 1)
Validation y shape: (523, 64, 128)
Test X shape: (10022, 64, 128, 1)
Input shape: (64, 128, 1)
Number of classes: 5


## Definisco modello

In [5]:
def encoder_block(input_tensor, num_filters):
    # Primo layer convoluzionale
    x = tfkl.Conv2D(num_filters, (3, 3), activation='relu', padding='same')(input_tensor)
    # Secondo layer convoluzionale
    x = tfkl.Conv2D(num_filters, (3, 3), activation='relu', padding='same')(x)
    # MaxPooling per downsampling
    p = tfkl.MaxPooling2D((2, 2))(x)
    return x, p

In [6]:
def decoder_block(input_tensor, concat_tensor, num_filters):
    # Upsampling (upscaling) della feature map
    x = tfkl.Conv2DTranspose(num_filters, (3, 3), strides=(2, 2), padding='same')(input_tensor)
    # Concatenazione con l'output dell'encoder corrispondente
    x = tfkl.concatenate([x, concat_tensor])
    # Convoluzione dopo la concatenazione
    x = tfkl.Conv2D(num_filters, (3, 3), activation='relu', padding='same')(x)
    x = tfkl.Conv2D(num_filters, (3, 3), activation='relu', padding='same')(x)
    return x

In [7]:
def unet(input_shape=(64, 128, 1), num_classes=5):
    inputs = tfkl.Input(shape=input_shape)

    # Encoder (downsampling)
    e1, p1 = encoder_block(inputs, 64)
    e2, p2 = encoder_block(p1, 128)
    e3, p3 = encoder_block(p2, 256)
    e4, p4 = encoder_block(p3, 512)

    # Bottleneck (layer centrale)
    b1 = tfkl.Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    b1 = tfkl.Conv2D(1024, (3, 3), activation='relu', padding='same')(b1)

    # Decoder (upsampling)
    d1 = decoder_block(b1, e4, 512)
    d2 = decoder_block(d1, e3, 256)
    d3 = decoder_block(d2, e2, 128)
    d4 = decoder_block(d3, e1, 64)

    # Output layer with softmax activation for multi-class segmentation
    outputs = tfkl.Conv2D(num_classes, (1, 1), activation='softmax')(d4)

    # Crea il modello
    model = models.Model(inputs, outputs)

    return model

In [8]:
# Crea il modello U-Net
model = unet(input_shape=(64, 128, 1), num_classes=5)

# Define the MeanIoU ignoring the background class
mean_iou = tfk.metrics.MeanIoU(num_classes=num_classes, ignore_class=0, sparse_y_pred=False)

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=[mean_iou,'accuracy'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 64, 128, 1)     │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 64, 128, 64)    │            640 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 64, 128, 64)    │         36,928 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 32, 64, 64)     │              0 │ conv2d_1[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 32, 64, 128)    │         73,856 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 32, 64, 128)    │        147,584 │ conv2d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 16, 32, 128)    │              0 │ conv2d_3[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 16, 32, 256)    │        295,168 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 16, 32, 256)    │        590,080 │ conv2d_4[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_2           │ (None, 8, 16, 256)     │              0 │ conv2d_5[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, 8, 16, 512)     │      1,180,160 │ max_pooling2d_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 8, 16, 512)     │      2,359,808 │ conv2d_6[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_3           │ (None, 4, 8, 512)      │              0 │ conv2d_7[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_8 (Conv2D)         │ (None, 4, 8, 1024)     │      4,719,616 │ max_pooling2d_3[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_9 (Conv2D)         │ (None, 4, 8, 1024)     │      9,438,208 │ conv2d_8[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose          │ (None, 8, 16, 512)     │      4,719,104 │ conv2d_9[0][0]         │
│ (Conv2DTranspose)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 8, 16, 1024)    │              0 │ conv2d_transpose[0][0… │
│                      

 Total params: 34,512,453 (131.65 MB)

 Trainable params: 34,512,453 (131.65 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Setup callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_mean_io_u',
    mode='max',
    patience=30,
    restore_best_weights=True
)

In [14]:
model.fit(
    X_train, y_train,  # Dati di addestramento
    epochs=50,         # Numero massimo di epoche
    batch_size=32,     # Dimensione del batch
    validation_data=(X_val, y_val),  # Set di validazione
    callbacks=[early_stopping]       # Callback di early stopping
)

Epoch 1/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 16s 239ms/step - accuracy: 0.3304 - loss: 1.3655 - mean_io_u: 0.0886 - val_accuracy: 0.3535 - val_loss: 1.3619 - val_mean_io_u: 0.0818
Epoch 2/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 16s 239ms/step - accuracy: 0.3458 - loss: 1.3530 - mean_io_u: 0.0927 - val_accuracy: 0.3564 - val_loss: 1.3643 - val_mean_io_u: 0.0771
Epoch 3/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 20s 238ms/step - accuracy: 0.3517 - loss: 1.3467 - mean_io_u: 0.0826 - val_accuracy: 0.3569 - val_loss: 1.3501 - val_mean_io_u: 0.0760
Epoch 4/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 21s 242ms/step - accuracy: 0.3476 - loss: 1.3420 - mean_io_u: 0.0794 - val_accuracy: 0.3552 - val_loss: 1.3506 - val_mean_io_u: 0.0773
Epoch 5/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 16s 236ms/step - accuracy: 0.3491 - loss: 1.3417 - mean_io_u: 0.0921 - val_accuracy: 0.3559 - val_loss: 1.3477 - val_mean_io_u: 0.0752
Epoch 6/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 20s 234ms/step - accuracy: 0.3535 - loss: 1.3344 - mean_io_u: 0.0879 - val_accuracy: 0.3557 - val

In [15]:
timestep_str = datetime.now().strftime("%y%m%d_%H%M%S")
model_filename = f"model_{timestep_str}.keras"
model.save(model_filename)
del model

print(f"Model saved to {model_filename}")

Model saved to model_241129_203210.keras


## 📊 Prepare Your Submission

In our Kaggle competition, submissions are made as `csv` files. To create a proper `csv` file, you need to flatten your predictions and include an `id` column as the first column of your dataframe. To maintain consistency between your results and our solution, please avoid shuffling the test set. The code below demonstrates how to prepare the `csv` file from your model predictions.




In [16]:
# If model_filename is not defined, load the most recent model from Google Drive
if model_filename not in globals() or model_filename is None:
    files = [f for f in os.listdir('.') if os.path.isfile(f) and f.startswith('model_') and f.endswith('.keras')]
    files.sort(key=lambda x: os.path.getmtime(x), reverse=True)
    if files:
        model_filename = files[0]
    else:
        raise FileNotFoundError("No model files found in the current directory.")

In [17]:
model = tfk.models.load_model(model_filename)
print(f"Model loaded from {model_filename}")

Model loaded from model_241129_203210.keras


In [18]:
preds = model.predict(X_test)
preds = np.argmax(preds, axis=-1)
print(f"Predictions shape: {preds.shape}")

314/314 ━━━━━━━━━━━━━━━━━━━━ 28s 87ms/step
Predictions shape: (10022, 64, 128)


In [19]:
def y_to_df(y) -> pd.DataFrame:
    """Converts segmentation predictions into a DataFrame format for Kaggle."""
    n_samples = len(y)
    y_flat = y.reshape(n_samples, -1)
    df = pd.DataFrame(y_flat)
    df["id"] = np.arange(n_samples)
    cols = ["id"] + [col for col in df.columns if col != "id"]
    return df[cols]

In [20]:
# Create and download the csv submission file
timestep_str = model_filename.replace("model_", "").replace(".keras", "")
submission_filename = f"submission_{timestep_str}.csv"
submission_df = y_to_df(preds)
submission_df.to_csv(submission_filename, index=False)

from google.colab import files
files.download(submission_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#  
<img src="https://airlab.deib.polimi.it/wp-content/uploads/2019/07/airlab-logo-new_cropped.png" width="350">

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/95/Instagram_logo_2022.svg/800px-Instagram_logo_2022.svg.png" width="15"> **Instagram:** https://www.instagram.com/airlab_polimi/

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/81/LinkedIn_icon.svg/2048px-LinkedIn_icon.svg.png" width="15"> **LinkedIn:** https://www.linkedin.com/company/airlab-polimi/
___
Credits: Alberto Archetti 📧 alberto.archetti@polito.it





```
   Copyright 2024 Alberto Archetti

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.
```